# Experiment 1

## Merging results

In [ ]:
import pandas as pd
import os

class HorizonMerger:
    def __init__(self, base_dir='Data/Test', models=['bitcn', 'tide', 'tft', 'nhits'],
                 model_version='model0_1', horizons=[1], prefixes=[None]):
        """
        Initializes the HorizonMerger with specified parameters.

        Parameters:
        - base_dir (str): Base directory where data is stored.
        - models (list): List of model names to process.
        - model_version (str): Version identifier for the models.
        - horizons (list): List of horizon values to process.
        - prefixes (list): List of prefixes to process, e.g., ['', '8TY'].
        """
        self.base_dir = base_dir
        self.models = models
        self.model_version = model_version
        self.horizons = horizons
        self.prefixes = prefixes  # List of prefixes
        # Mapping from model names to their corresponding column prefixes
        self.model_column_mapping = {
            'bitcn': 'AutoBiTCN',
            'tide': 'AutoTiDE',
            'tft': 'AutoTFT',
            'nhits': 'AutoNHITS'
        }

    def merge_models_for_horizon_and_prefix(self, horizon, prefix):
        """
        Merges CSV files for a specific horizon and prefix, then saves the full and subset data.

        Parameters:
        - horizon (int): The horizon value to process.
        - prefix (str): The prefix to add to the filename.
        """
        # List to store individual DataFrames
        df_list = []

        # Path to the current horizon's directory
        horizon_dir = os.path.join(self.base_dir, f'horizon_{horizon}')

        # Ensure the horizon directory exists
        if not os.path.isdir(horizon_dir):
            print(f"⚠️ Horizon directory does not exist: {horizon_dir}. Creating it...")
            os.makedirs(horizon_dir, exist_ok=True)

        # Read each model's CSV for the current horizon
        for model in self.models:
            # Update file name to include the prefix
            file_name = f'{prefix or ""}{model}_{self.model_version}_horizon_{horizon}.csv'
            file_path = os.path.join(horizon_dir, file_name)
            if not os.path.exists(file_path):
                print(f"❌ File not found: {file_path}")
                return  # Skip processing this horizon if any file is missing
            df = pd.read_csv(file_path)
            df_list.append(df)
            print(f"✅ Loaded {file_path}")

        # Merge all DataFrames on the specified keys
        print(f"🔗 Merging data for horizon {horizon} with prefix '{prefix}'...")
        merged_df = df_list[0]
        for df in df_list[1:]:
            merged_df = pd.merge(merged_df, df, on=['unique_id', 'ds', 'cutoff', 'y'], how='inner')
        print("✅ Merging completed.")

        # Define the output filename and path for the full merged data
        full_output_filename = f'{prefix or ""}{"_".join(self.models)}_{self.model_version}_full_horizon_{horizon}.csv'
        full_output_path = os.path.join(horizon_dir, full_output_filename)

        # Save the merged DataFrame
        merged_df.to_csv(full_output_path, index=False)
        print(f"💾 Saved merged data to {full_output_path}")

        # Prepare subset columns based on the mapping
        subset_columns = ['unique_id', 'ds', 'y']
        for model in self.models:
            col_prefix = self.model_column_mapping.get(model)
            if col_prefix:
                subset_columns.extend([col_prefix, f'{col_prefix}1'])
            else:
                print(f"⚠️ No column mapping found for model '{model}'. Skipping its subset columns.")

        # Check for missing columns and handle them
        missing_cols = [col for col in subset_columns if col not in merged_df.columns]
        if missing_cols:
            print(f"⚠️ Missing columns for subset: {missing_cols}")
            for col in missing_cols:
                merged_df[col] = pd.NA

        # Create the subset DataFrame
        df_subset = merged_df[subset_columns].copy()

        # Define the output filename and path for the subset data
        subset_output_filename = f'{prefix or ""}{"_".join(self.models)}_{self.model_version}_sub_horizon_{horizon}.csv'
        subset_output_path = os.path.join(horizon_dir, subset_output_filename)

        # Save the subset DataFrame
        df_subset.to_csv(subset_output_path, index=False)
        print(f"💾 Saved subset data to {subset_output_path}\n")

    def process_all_horizons_and_prefixes(self):
        """
        Processes all specified horizons and prefixes by merging and saving their respective data.
        """
        for prefix in self.prefixes:
            for horizon in self.horizons:
                print(f"🚀 Processing horizon {horizon} with prefix '{prefix}'...")
                self.merge_models_for_horizon_and_prefix(horizon, prefix)
                print(f"✅ Finished processing horizon {horizon} for prefix '{prefix}'\n")

if __name__ == "__main__":
    # Initialize the HorizonMerger with horizons and prefixes
    merger = HorizonMerger(
        horizons=[1, 5, 10, 20],
        prefixes=[None, '8TY']
    )

    # Start processing all specified horizons and prefixes
    merger.process_all_horizons_and_prefixes()
    print("✅ Completed all processing")

🚀 Processing horizon 1 with prefix 'None'...
✅ Loaded Data/Test/horizon_1/bitcn_model0_1_horizon_1.csv
✅ Loaded Data/Test/horizon_1/tide_model0_1_horizon_1.csv
✅ Loaded Data/Test/horizon_1/tft_model0_1_horizon_1.csv
✅ Loaded Data/Test/horizon_1/nhits_model0_1_horizon_1.csv
🔗 Merging data for horizon 1 with prefix 'None'...
✅ Merging completed.
💾 Saved merged data to Data/Test/horizon_1/bitcn_tide_tft_nhits_model0_1_full_horizon_1.csv
💾 Saved subset data to Data/Test/horizon_1/bitcn_tide_tft_nhits_model0_1_sub_horizon_1.csv

✅ Finished processing horizon 1 for prefix 'None'

🚀 Processing horizon 5 with prefix 'None'...
✅ Loaded Data/Test/horizon_5/bitcn_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/tide_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/tft_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/nhits_model0_1_horizon_5.csv
🔗 Merging data for horizon 5 with prefix 'None'...
✅ Merging completed.
💾 Saved merged data to Data/Test/horizon_5/bitcn_tide_tft_nhits_model0_1_

## QLIKE evaluation and DM test

Following the evaluation metrics as well as MCS and DM test from Suoto and Moradi 2024

https://www.emerald.com/insight/content/doi/10.1108/cfri-01-2024-0032/full/html

https://github.com/hugogobato/Can-Transformers-Transform-Financial-Forecasting-

In [ ]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import t
from arch.bootstrap import MCS

# Define the horizons and prefixes
horizons = [1, 5, 10, 20]
prefixes = ['', '8TY']

# Loop over each prefix and horizon
for prefix in prefixes:
    for horizon in horizons:
        print(f"Processing Prefix: '{prefix}', Horizon: {horizon}")

        # Define file paths based on prefix and horizon
        prefix_str = f"{prefix}_" if prefix else ''
        file_name = f"{prefix}bitcn_tide_tft_nhits_model0_1_sub_horizon_{horizon}.csv"
        data_path = f"Data/Test/horizon_{horizon}/{file_name}"
        output_dir = f"Data/Evaluation/horizon_{horizon}"
        os.makedirs(output_dir, exist_ok=True)

        # Read the CSV file
        df = pd.read_csv(data_path)
        df = df.sort_values(by=['unique_id', 'ds']).drop_duplicates(subset=['ds', 'unique_id'])
        df['ds'] = pd.to_datetime(df['ds'])

        # Extract model columns
        model_columns = ['AutoTiDE', 'AutoTiDE1', 'AutoTFT', 'AutoTFT1',
                         'AutoBiTCN', 'AutoBiTCN1', 'AutoNHITS', 'AutoNHITS1']

        # Create a dictionary to hold pivoted DataFrames for each model
        model_dfs = {}
        for model in model_columns:
            model_df = df.pivot(index='ds', columns='unique_id', values=model)
            model_dfs[model] = model_df

        # Pivot actual values
        Actuals = df.pivot(index='ds', columns='unique_id', values='y')

        # Calculate error metrics for each model
        metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
        error_metrics = {metric: {} for metric in metrics}

        for model_name, predictions in model_dfs.items():
            # Align the predictions and actuals
            predictions, actuals = predictions.align(Actuals, join='inner', axis=0)
            errors = predictions - actuals

            # RMSE Calculation
            mse = (errors ** 2).mean().mean()
            error_metrics['RMSE'][model_name] = np.sqrt(mse)

            # MAE Calculation
            mae = errors.abs().mean().mean()
            error_metrics['MAE'][model_name] = mae

            # MAPE Calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                mape = (errors.abs() / actuals).replace([np.inf, -np.inf], np.nan).mean().mean()
            error_metrics['MAPE'][model_name] = mape

            # QLIKE Calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                ratio = actuals / predictions
                qlike = (ratio - np.log(ratio) - 1).replace([np.inf, -np.inf], np.nan).mean().mean()
            error_metrics['QLIKE'][model_name] = qlike

        # Create a DataFrame with error metrics
        error_metrics_df = pd.DataFrame(error_metrics)
        metrics_file = f"{output_dir}/{prefix_str}metrics_horizon_{horizon}.csv"
        error_metrics_df.to_csv(metrics_file)

        # Function to calculate losses for MCS
        def calculate_losses(metric, model_dfs, actuals):
            residuals_dict = {}
            for model_name, predictions in model_dfs.items():
                # Align predictions and actuals
                predictions_aligned, actuals_aligned = predictions.align(actuals, join='inner', axis=0)
                if metric == 'RMSE':
                    residuals = (predictions_aligned - actuals_aligned) ** 2
                elif metric == 'MAE':
                    residuals = (predictions_aligned - actuals_aligned).abs()
                elif metric == 'MAPE':
                    with np.errstate(divide='ignore', invalid='ignore'):
                        residuals = (predictions_aligned - actuals_aligned).abs() / actuals_aligned
                elif metric == 'QLIKE':
                    with np.errstate(divide='ignore', invalid='ignore'):
                        ratio = actuals_aligned / predictions_aligned
                        residuals = ratio - np.log(ratio) - 1
                else:
                    raise ValueError("Unsupported metric")

                residuals = residuals.replace([np.inf, -np.inf], np.nan)
                residuals_dict[model_name] = residuals.mean(axis=1)

            return pd.DataFrame(residuals_dict).dropna()

        # Run MCS procedure for each metric
        pvalues_list = []

        for metric in metrics:
            losses = calculate_losses(metric, model_dfs, Actuals)
            mcs = MCS(losses, size=0.05, method="R", block_size=1000)
            mcs.compute()
            pvalues = mcs.pvalues.reset_index().rename(columns={'Pvalue': f'Pvalue_{metric}'})
            pvalues_list.append(pvalues)

        # Merge p-values and save
        merged_pvalues = pvalues_list[0]
        for pvalues in pvalues_list[1:]:
            merged_pvalues = merged_pvalues.merge(pvalues, on='Model name', how='outer')

        mcs_file = f"{output_dir}/{prefix_str}MCS_horizon_{horizon}.csv"
        merged_pvalues.to_csv(mcs_file, index=False)

        # Function for Diebold-Mariano test
        def dm_test(actual, pred1, pred2, h=1, crit="RMSE"):
            actual, pred1, pred2 = map(np.array, [actual, pred1, pred2])
            T = len(actual)

            # Ensure predictions and actuals are of the same length
            if len(actual) != len(pred1) or len(actual) != len(pred2):
                return np.nan

            # Remove NaN values
            mask = ~np.isnan(actual) & ~np.isnan(pred1) & ~np.isnan(pred2)
            actual = actual[mask]
            pred1 = pred1[mask]
            pred2 = pred2[mask]
            T = len(actual)
            if T == 0:
                return np.nan

            if crit == "RMSE":
                loss_diff = (actual - pred1) ** 2 - (actual - pred2) ** 2
            elif crit == "MAE":
                loss_diff = np.abs(actual - pred1) - np.abs(actual - pred2)
            elif crit == "MAPE":
                with np.errstate(divide='ignore', invalid='ignore'):
                    loss_diff = (np.abs(actual - pred1) / actual) - (np.abs(actual - pred2) / actual)
            elif crit == "QLIKE":
                with np.errstate(divide='ignore', invalid='ignore'):
                    loss1 = actual / np.abs(pred1) - np.log(actual / np.abs(pred1)) - 1
                    loss2 = actual / np.abs(pred2) - np.log(actual / np.abs(pred2)) - 1
                    loss_diff = loss1 - loss2
            else:
                raise ValueError("Unsupported criterion")

            loss_diff = loss_diff[~np.isnan(loss_diff)]
            if len(loss_diff) == 0:
                return np.nan

            d_mean = np.mean(loss_diff)
            gamma = [np.correlate(loss_diff - d_mean, loss_diff - d_mean, 'full')[len(loss_diff)-1:] / len(loss_diff)][0]
            V_d = gamma[0] + 2 * sum(gamma[1:h])
            DM_stat = d_mean / np.sqrt(V_d / len(loss_diff))
            adj = ((len(loss_diff) + 1 - 2 * h + h * (h - 1) / len(loss_diff)) / len(loss_diff)) ** 0.5
            return DM_stat * adj

        # Run DM tests and save results
        models = model_columns
        stocks = Actuals.columns

        for metric in metrics:
            better_count_matrix = pd.DataFrame(0, index=models, columns=models)

            for model_a, model_b in combinations(models, 2):
                counts = {'a_better': 0, 'b_better': 0}
                for stock in stocks:
                    actual = Actuals[stock]
                    pred1 = model_dfs[model_a][stock]
                    pred2 = model_dfs[model_b][stock]
                    combined = pd.concat([actual, pred1, pred2], axis=1).dropna()

                    if combined.empty:
                        continue

                    dm_stat = dm_test(combined.iloc[:, 0], combined.iloc[:, 1], combined.iloc[:, 2], h=1, crit=metric)
                    if np.isnan(dm_stat):
                        continue
                    if dm_stat > 1.96:
                        counts['a_better'] += 1
                    elif dm_stat < -1.96:
                        counts['b_better'] += 1

                better_count_matrix.loc[model_a, model_b] = counts['a_better']
                better_count_matrix.loc[model_b, model_a] = counts['b_better']

            # Summarize results
            better_count_matrix['Outperform Count'] = better_count_matrix.sum(axis=1)
            better_count_matrix.loc['Outperformed Count'] = better_count_matrix.sum(axis=0)
            better_count_matrix.loc['Outperformed Count', 'Outperform Count'] = np.nan
            dm_file = f"{output_dir}/{prefix_str}DM_{metric}_horizon_{horizon}.csv"
            better_count_matrix.to_csv(dm_file)

        print(f"Completed processing for Prefix: '{prefix}', Horizon: {horizon}\n")

Processing Prefix: '', Horizon: 1
Completed processing for Prefix: '', Horizon: 1

Processing Prefix: '', Horizon: 5
Completed processing for Prefix: '', Horizon: 5

Processing Prefix: '', Horizon: 10
Completed processing for Prefix: '', Horizon: 10

Processing Prefix: '', Horizon: 20
Completed processing for Prefix: '', Horizon: 20

Processing Prefix: '8TY', Horizon: 1
Completed processing for Prefix: '8TY', Horizon: 1

Processing Prefix: '8TY', Horizon: 5
Completed processing for Prefix: '8TY', Horizon: 5

Processing Prefix: '8TY', Horizon: 10
Completed processing for Prefix: '8TY', Horizon: 10

Processing Prefix: '8TY', Horizon: 20
Completed processing for Prefix: '8TY', Horizon: 20



## Evaluation agregation

Following the evaluation metrics as well as MCS and DM test from Suoto and Moradi 2024

https://www.emerald.com/insight/content/doi/10.1108/cfri-01-2024-0032/full/html

https://github.com/hugogobato/Can-Transformers-Transform-Financial-Forecasting-

### Metrics

In [ ]:
import pandas as pd

# Load the data for each horizon and merge by metric
horizons = [1, 5, 10, 20]
metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
prefixes = ['', '8TY_']

metric_data = {metric: [] for metric in metrics}

# Process each horizon, prefix, and add to the corresponding metric dictionary
for prefix in prefixes:
    for horizon in horizons:
        # Adjust file path with prefix
        file_path = f'Data/Evaluation/horizon_{horizon}/{prefix}metrics_horizon_{horizon}.csv'
        try:
            df = pd.read_csv(file_path)
            df = df.rename(columns={'Unnamed: 0': 'model'})
            df['model'] = df['model'].str.replace('Auto', '', regex=False)
            for metric in metrics:
                metric_df = df[['model', metric]].copy()
                metric_df = metric_df.rename(columns={metric: f'{metric}_{prefix}horizon_{horizon}'})
                metric_data[metric].append(metric_df)
        except FileNotFoundError:
            print(f"File not found: {file_path}. Skipping this file.")

# Create the output directory if it doesn't exist
output_dir = 'Data/Evaluation/Final'
os.makedirs(output_dir, exist_ok=True)

# Export each metric DataFrame to an Excel file
with pd.ExcelWriter(f'{output_dir}/metrics_across_horizons_and_prefixes.xlsx') as writer:
    for metric in metrics:
        # Concatenate the metric dataframes for each metric across horizons
        if metric_data[metric]:  # Check if data exists for the metric
            result_df = pd.concat(metric_data[metric], axis=1)
            result_df = result_df.loc[:,~result_df.columns.duplicated()]  # remove duplicate 'model' columns
            # Write each metric to a separate sheet in the Excel file
            result_df.to_excel(writer, sheet_name=metric, index=False)

"Excel file with metrics across horizons and prefixes has been created in 'Data/Evaluation/Final/'."

"Excel file with metrics across horizons and prefixes has been created in 'Data/Evaluation/Final/'."

### MCS

In [ ]:
import pandas as pd

# Load the data for each horizon and merge by metric
horizons = [1, 5, 10, 20]
metrics = ['Pvalue_RMSE', 'Pvalue_MAE',	'Pvalue_MAPE',	'Pvalue_QLIKE']
prefixes = ['', '8TY_']

metric_data = {metric: [] for metric in metrics}

# Process each horizon, prefix, and add to the corresponding metric dictionary
for prefix in prefixes:
    for horizon in horizons:
        # Adjust file path with prefix
        file_path = f'Data/Evaluation/horizon_{horizon}/{prefix}MCS_horizon_{horizon}.csv'
        try:
            df = pd.read_csv(file_path)
            df = df.rename(columns={'Model name': 'model'})
            df['model'] = df['model'].str.replace('Auto', '', regex=False)
            for metric in metrics:
                metric_df = df[['model', metric]].copy()
                metric_df = metric_df.rename(columns={metric: f'{metric}_{prefix}horizon_{horizon}'})
                metric_data[metric].append(metric_df)
        except FileNotFoundError:
            print(f"File not found: {file_path}. Skipping this file.")

output_dir = 'Data/Evaluation/Final'
os.makedirs(output_dir, exist_ok=True)

# Export each metric DataFrame to an Excel file
with pd.ExcelWriter(f'{output_dir}/MCS_across_horizons_and_prefixes.xlsx') as writer:
    for metric in metrics:
        # Concatenate the metric dataframes for each metric across horizons
        if metric_data[metric]:  # Check if data exists for the metric
            result_df = pd.concat(metric_data[metric], axis=1)
            result_df = result_df.loc[:,~result_df.columns.duplicated()]  # remove duplicate 'model' columns
            # Write each metric to a separate sheet in the Excel file
            result_df.to_excel(writer, sheet_name=metric, index=False)
"Excel file with MCS across horizons and prefixes has been created in 'Data/Evaluation/Final/'."

"Excel file with MCS across horizons and prefixes has been created in 'Data/Evaluation/Final/'."

### DM

In [ ]:
import pandas as pd
import os

# Initialize dictionaries to store data for each metric and horizon
metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
prefixes = ['', '8TY_']
horizons = [1, 5, 10, 20]
metric_data = {metric: [] for metric in metrics}

# Process each metric and compile data across all horizons and prefixes
for metric in metrics:
    for prefix in prefixes:
        for horizon in horizons:
            # Define file path and check if it exists
            file_path = f"Data/Evaluation/horizon_{horizon}/{prefix}DM_{metric}_horizon_{horizon}.csv"
            if os.path.exists(file_path):
                # Read the data, rename columns, and filter as specified
                df = pd.read_csv(file_path)
                df = df.rename(columns={'Unnamed: 0': 'model', 'Outperform Count': f'{metric}_{prefix.strip("_")}_h{horizon}_Outperform'})
                df = df[df['model'] != 'Outperformed Count']
                df = df[['model', f'{metric}_{prefix.strip("_")}_h{horizon}_Outperform']]
                df['model'] = df['model'].str.replace('Auto', '', regex=False)
                metric_data[metric].append(df)

# Save data for each metric into an Excel file with each horizon's data in one sheet per metric
with pd.ExcelWriter('Data/Evaluation/Final/DM_across_horizons_and_prefixes.xlsx') as writer:
    for metric, data_frames in metric_data.items():
        # Concatenate data frames for the specific metric along columns by 'model' as the index
        result_df = pd.concat(data_frames, axis=1).loc[:,~pd.concat(data_frames, axis=1).columns.duplicated()]
        result_df.to_excel(writer, sheet_name=metric, index=False)

'Excel file created with summary of all metrics.'

'Excel file created with summary of all metrics.'